In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

class Estrattore:
  def __init__(self, data: DataFrame):
    self.data = data.dropna()

  def get_df(self):
    return self.data

  def get_colonne(self, colonne: List[str]):
    return self.data[colonne]
  
  def get_nomi_regioni(self):
    return self.data["Regione"].unique()
  
  def get_regione_by_nome(self, regioneRic: str):
    return self.data[self.data.Regione==regioneRic]
  
  def get_regione_by_anno(self, annoRic: int):
    return self.data[self.data.Anno==annoRic]
  
  def get_dati(self, regione: str, anno: int) -> Tuple[int, int]:
        df_filtered = self.data[(self.data['Regione'] == regione) & (self.data['Anno'] == anno)]
        if not df_filtered.empty:
            numero_clienti = df_filtered['Arrivi'].values[0]
            numero_notti = df_filtered['Presenze'].values[0]
            return numero_clienti, numero_notti
        return 0, 0

# URLs for the datasets
URL_DATA = [
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-agriturismi-in-Italia-per-regione.csv",
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv",
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-agriturismi-in-Italia-per-regione.csv",
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
    "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv"
]

# Load datasets
df_arrivi_agriturismi = pd.read_csv(URL_DATA[0], delimiter=';')
df_presenze_agriturismi = pd.read_csv(URL_DATA[3], delimiter=';')
df_arrivi_alberghi = pd.read_csv(URL_DATA[1], delimiter=';')
df_presenze_alberghi = pd.read_csv(URL_DATA[4], delimiter=';')
df_arrivi_campeggi = pd.read_csv(URL_DATA[2], delimiter=';')
df_presenze_campeggi = pd.read_csv(URL_DATA[5], delimiter=';')

# Merge dataframes for each category
df_agriturismi = pd.merge(df_arrivi_agriturismi, df_presenze_agriturismi, on=['Regione', 'Anno'], how='inner')
df_alberghi = pd.merge(df_arrivi_alberghi, df_presenze_alberghi, on=['Regione', 'Anno'], how='inner')
df_campeggi = pd.merge(df_arrivi_campeggi, df_presenze_campeggi, on=['Regione', 'Anno'], how='inner')

e_agriturismi = Estrattore(df_agriturismi)
e_campeggi = Estrattore(df_campeggi)
e_alberghi = Estrattore(df_alberghi)

dati_per_regione = {}

regioni = e_agriturismi.get_nomi_regioni()
anni = [2008, 2009, 2010, 2011]
categorie = ["Agriturismi", "Campeggi", "Alberghi"]

for regione in regioni:
    dati_per_regione[regione] = {}
    for categoria in categorie:
        dati_per_regione[regione][categoria] = {}
        for anno in anni:
            if categoria == "Agriturismi":
                numero_clienti, numero_notti = e_agriturismi.get_dati(regione, anno)
            elif categoria == "Campeggi":
                numero_clienti, numero_notti = e_campeggi.get_dati(regione, anno)
            elif categoria == "Alberghi":
                numero_clienti, numero_notti = e_alberghi.get_dati(regione, anno)
            dati_per_regione[regione][categoria][anno] = (numero_clienti, numero_notti)
   

def plot_dati_per_categoria(regione, categoria):
    anni = list(dati_per_regione[regione][categoria].keys())
    arrivi = [dati_per_regione[regione][categoria][anno][0] for anno in anni]
    presenze = [dati_per_regione[regione][categoria][anno][1] for anno in anni]
    
    plt.figure(figsize=(10, 5))
    plt.plot(anni, arrivi, marker='o', label='Arrivi')
    plt.plot(anni, presenze, marker='o', label='Presenze')
    plt.title(f'{categoria} in {regione}')
    plt.xlabel('Anno')
    plt.ylabel('Numero')
    plt.legend()
    plt.grid(True)
    plt.show()

# Esempio di utilizzo
plot_dati_per_categoria('Piemonte', 'Agriturismi')
# plot_dati_per_categoria('Piemonte', 'Alberghi')
# plot_dati_per_categoria('Piemonte', 'Campeggi')


: 